**SFT** CODE **bold text**

In [10]:
import pandas as pd

df = pd.read_csv("aapl_2022_updated.csv")
print(df.columns)


Index(['Date', 'Close Price', 'High Price', 'Low Price', 'Open Price',
       'Trading Volume'],
      dtype='object')


In [11]:
#SFT CODE
import pandas as pd
import numpy as np

# 1️⃣ Load your dataset
file_path = "aapl_2022_updated.csv"
df = pd.read_csv(file_path)

# Inspect the dataset
print(df.head())
print(df.info())

# 2️⃣ Function to generate SFT-style output per row
def sft_generate(row):
    # Simulate predicted_close: small random variation around actual Close Price
    predicted_close = row['Close Price'] * (1 + np.random.uniform(-0.01, 0.01))

    # Determine trend from Open Price vs Close Price
    if row['Close Price'] > row['Open Price'] * 1.005:
        trend = "Bullish"
    elif row['Close Price'] < row['Open Price'] * 0.995:
        trend = "Bearish"
    else:
        trend = "Neutral"

    # Simulate sentiment score (0.0 - 1.0)
    sentiment_score = np.random.uniform(0, 1)

    return {
        "date": row['Date'],
        "predicted_close": round(predicted_close, 2),
        "trend": trend,
        "sentiment_score": round(sentiment_score, 2)
    }

# 3️⃣ Apply function to each row
sft_output = df.apply(sft_generate, axis=1)

# Convert to DataFrame for better viewing
sft_df = pd.DataFrame(list(sft_output))

# View first 5 rows
print(sft_df.head())

# 4️⃣ Optional: save SFT output to CSV
sft_df.to_csv("aapl_sft_output.csv", index=False)
print("SFT output saved to aapl_sft_output.csv")


         Date  Close Price  High Price   Low Price  Open Price  Trading Volume
0  03-01-2022   178.645645  179.499574  174.425140  174.542917       104487900
1  04-01-2022   176.378311  179.558426  175.809030  179.254159        99310400
2  05-01-2022   171.686722  176.839679  171.411899  176.290033        94537600
3  06-01-2022   168.820679  172.059683  168.467332  169.507737        96904000
4  07-01-2022   168.987549  170.921136  167.868622  169.694241        86709100
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            251 non-null    object 
 1   Close Price     251 non-null    float64
 2   High Price      251 non-null    float64
 3   Low Price       251 non-null    float64
 4   Open Price      251 non-null    float64
 5   Trading Volume  251 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 11.9+ KB

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load dataset
df = pd.read_csv("aapl_2022_updated.csv")

# Convert Date to datetime if needed
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Prepare regression
df['Day_Index'] = np.arange(len(df))  # time index for regression
X = df['Day_Index'].values.reshape(-1, 1)
y = df['Close Price'].values

# Train linear regression
model = LinearRegression()
model.fit(X, y)

# Predict Close Price (next day)
df['predicted_close'] = model.predict(X)


In [13]:
# Trend based on predicted_close vs Open Price
def calculate_trend(row):
    if row['predicted_close'] > row['Open Price'] * 1.002:
        return "Bullish"
    elif row['predicted_close'] < row['Open Price'] * 0.998:
        return "Bearish"
    else:
        return "Neutral"

# Sentiment_score based on normalized predicted change
def calculate_sentiment(row):
    change_pct = (row['predicted_close'] - row['Open Price']) / row['Open Price']
    sentiment = (change_pct + 0.05) / 0.1  # scale roughly to 0-1
    return min(max(sentiment, 0), 1)  # ensure 0 <= sentiment <= 1

df['trend'] = df.apply(calculate_trend, axis=1)
df['sentiment_score'] = df.apply(calculate_sentiment, axis=1)


In [14]:
# Daily volatility = High - Low
df['volatility'] = df['High Price'] - df['Low Price']

# Simple Moving Average (SMA) of Close Price over 5 days
df['SMA_5'] = df['Close Price'].rolling(window=5).mean()

# Simple Moving Average (SMA) over 10 days
df['SMA_10'] = df['Close Price'].rolling(window=10).mean()

# Optional: 5-day return %
df['return_5d'] = df['Close Price'].pct_change(periods=5)


In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load dataset
df = pd.read_csv("aapl_2022_updated.csv")
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Regression-based predicted_close
df['Day_Index'] = np.arange(len(df))
X = df['Day_Index'].values.reshape(-1, 1)
y = df['Close Price'].values
model = LinearRegression()
model.fit(X, y)
df['predicted_close'] = model.predict(X)

# Trend
def calculate_trend(row):
    if row['predicted_close'] > row['Open Price'] * 1.002:
        return "Bullish"
    elif row['predicted_close'] < row['Open Price'] * 0.998:
        return "Bearish"
    else:
        return "Neutral"

df['trend'] = df.apply(calculate_trend, axis=1)

# Sentiment
def calculate_sentiment(row):
    change_pct = (row['predicted_close'] - row['Open Price']) / row['Open Price']
    sentiment = (change_pct + 0.05) / 0.1
    return min(max(sentiment, 0), 1)

df['sentiment_score'] = df.apply(calculate_sentiment, axis=1)

# Extra features
df['volatility'] = df['High Price'] - df['Low Price']
df['SMA_5'] = df['Close Price'].rolling(window=5).mean()
df['SMA_10'] = df['Close Price'].rolling(window=10).mean()
df['return_5d'] = df['Close Price'].pct_change(periods=5)

# Select final SFT output columns
sft_df = df[['Date', 'predicted_close', 'trend', 'sentiment_score',
             'volatility', 'SMA_5', 'SMA_10', 'return_5d']]

# Display the first 10 rows
print(sft_df.head(10))

# Save to CSV
sft_df.to_csv("aapl_sft_output_realistic.csv", index=False)
print("✅ Realistic SFT output saved to 'aapl_sft_output_realistic.csv'")


        Date  predicted_close    trend  sentiment_score  volatility  \
0 2022-01-03       166.433055  Bearish         0.035366    5.074434   
1 2022-01-04       166.320852  Bearish         0.000000    3.749396   
2 2022-01-05       166.208649  Bearish         0.000000    5.427781   
3 2022-01-06       166.096447  Bearish         0.298753    3.592351   
4 2022-01-07       165.984244  Bearish         0.281372    3.052514   
5 2022-01-10       165.872041  Neutral         0.495021    4.249965   
6 2022-01-11       165.759838  Bearish         0.300459    4.279394   
7 2022-01-12       165.647635  Bearish         0.082513    2.316362   
8 2022-01-13       165.535432  Bearish         0.094542    4.740723   
9 2022-01-14       165.423229  Bearish         0.336498    2.640279   

        SMA_5      SMA_10  return_5d  
0         NaN         NaN        NaN  
1         NaN         NaN        NaN  
2         NaN         NaN        NaN  
3         NaN         NaN        NaN  
4  172.903781         N

RL CODE **bold text**

In [16]:
#RL CODE
import pandas as pd
import numpy as np

# Load SFT dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Define state features
features = ['predicted_close', 'trend', 'sentiment_score', 'volatility', 'SMA_5', 'SMA_10', 'return_5d']

# Encode 'trend' as numeric
trend_map = {'Bullish': 1, 'Neutral': 0, 'Bearish': -1}
df['trend_num'] = df['trend'].map(trend_map)

# Fill NaNs with 0 for simplicity
df.fillna(0, inplace=True)

# Build simplified states array
states = df[['predicted_close', 'trend_num', 'sentiment_score',
             'volatility', 'SMA_5', 'SMA_10', 'return_5d']].values

# RL Q-Learning setup
num_actions = 3  # 0=Hold,1=Buy,2=Sell
Q = np.zeros((len(states), num_actions))
alpha = 0.1  # learning rate
gamma = 0.95  # discount factor
epsilon = 0.1  # exploration

# Simulate prices for rewards
prices = df['predicted_close'].values

# Q-learning loop
for t in range(len(states)-1):
    if np.random.rand() < epsilon:
        action = np.random.randint(num_actions)
    else:
        action = np.argmax(Q[t])

    # Compute reward: profit/loss
    if action == 1:  # Buy
        reward = prices[t+1] - prices[t]
    elif action == 2:  # Sell
        reward = prices[t] - prices[t+1]
    else:  # Hold
        reward = 0

    # Update Q-table
    Q[t, action] = Q[t, action] + alpha * (reward + gamma * np.max(Q[t+1]) - Q[t, action])

# Final policy
policy = np.argmax(Q, axis=1)

# Add policy/actions to dataset
df['action'] = policy
df['action_label'] = df['action'].map({0:'Hold', 1:'Buy', 2:'Sell'})

# View first 10 steps
print(df[['Date', 'predicted_close', 'trend', 'sentiment_score', 'action_label']].head(10))


         Date  predicted_close    trend  sentiment_score action_label
0  2022-01-03       166.433055  Bearish         0.035366         Hold
1  2022-01-04       166.320852  Bearish         0.000000         Hold
2  2022-01-05       166.208649  Bearish         0.000000         Hold
3  2022-01-06       166.096447  Bearish         0.298753         Hold
4  2022-01-07       165.984244  Bearish         0.281372         Hold
5  2022-01-10       165.872041  Neutral         0.495021         Hold
6  2022-01-11       165.759838  Bearish         0.300459         Hold
7  2022-01-12       165.647635  Bearish         0.082513         Hold
8  2022-01-13       165.535432  Bearish         0.094542         Hold
9  2022-01-14       165.423229  Bearish         0.336498         Hold


In [17]:
import pandas as pd

# Load SFT dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Initialize action column
df['action_label'] = 'Hold'

# Parameters for Buy/Sell thresholds (1‰ = 0.1% change)
buy_threshold = 0.001
sell_threshold = -0.001

# Simulate actions based on predicted_close change
for i in range(1, len(df)):
    change_pct = (df.loc[i, 'predicted_close'] - df.loc[i-1, 'predicted_close']) / df.loc[i-1, 'predicted_close']
    if change_pct > buy_threshold:
        df.loc[i, 'action_label'] = 'Buy'
    elif change_pct < sell_threshold:
        df.loc[i, 'action_label'] = 'Sell'
    else:
        df.loc[i, 'action_label'] = 'Hold'

# Simulate cumulative profit
initial_cash = 10000  # start with $10,000
cash = initial_cash
position = 0  # number of shares held
portfolio_values = []

for i in range(len(df)):
    price = df.loc[i, 'predicted_close']
    action = df.loc[i, 'action_label']

    if action == 'Buy' and cash >= price:
        position += 1
        cash -= price
    elif action == 'Sell' and position > 0:
        position -= 1
        cash += price

    portfolio_value = cash + position * price
    portfolio_values.append(portfolio_value)

df['portfolio_value'] = portfolio_values

# View first 15 rows
print(df[['Date', 'predicted_close', 'action_label', 'portfolio_value']].head(15))

# Save to CSV
df.to_csv("aapl_rl_simulation.csv", index=False)
print("✅ RL simulation output saved to 'aapl_rl_simulation.csv'")


          Date  predicted_close action_label  portfolio_value
0   2022-01-03       166.433055         Hold          10000.0
1   2022-01-04       166.320852         Hold          10000.0
2   2022-01-05       166.208649         Hold          10000.0
3   2022-01-06       166.096447         Hold          10000.0
4   2022-01-07       165.984244         Hold          10000.0
5   2022-01-10       165.872041         Hold          10000.0
6   2022-01-11       165.759838         Hold          10000.0
7   2022-01-12       165.647635         Hold          10000.0
8   2022-01-13       165.535432         Hold          10000.0
9   2022-01-14       165.423229         Hold          10000.0
10  2022-01-18       165.311026         Hold          10000.0
11  2022-01-19       165.198824         Hold          10000.0
12  2022-01-20       165.086621         Hold          10000.0
13  2022-01-21       164.974418         Hold          10000.0
14  2022-01-24       164.862215         Hold          10000.0
✅ RL sim

In [18]:
import pandas as pd

# Load SFT dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Initialize action column
df['action_label'] = 'Hold'

# Use small absolute thresholds for price changes
buy_threshold = 0.01   # $0.01 increase triggers Buy
sell_threshold = -0.01 # $0.01 decrease triggers Sell

# Assign actions based on predicted_close change
for i in range(1, len(df)):
    change = df.loc[i, 'predicted_close'] - df.loc[i-1, 'predicted_close']
    if change > buy_threshold:
        df.loc[i, 'action_label'] = 'Buy'
    elif change < sell_threshold:
        df.loc[i, 'action_label'] = 'Sell'
    else:
        df.loc[i, 'action_label'] = 'Hold'

# Simulate portfolio value
initial_cash = 10000  # starting cash
cash = initial_cash
position = 0          # number of shares held
portfolio_values = []

for i in range(len(df)):
    price = df.loc[i, 'predicted_close']
    action = df.loc[i, 'action_label']

    if action == 'Buy' and cash >= price:
        position += 1
        cash -= price
    elif action == 'Sell' and position > 0:
        position -= 1
        cash += price

    portfolio_value = cash + position * price
    portfolio_values.append(portfolio_value)

df['portfolio_value'] = portfolio_values

# Display first 15 rows
print(df[['Date', 'predicted_close', 'action_label', 'portfolio_value']].head(15))

# Save to CSV
df.to_csv("aapl_rl_simulation_fixed.csv", index=False)
print("✅ Fixed RL simulation output saved to 'aapl_rl_simulation_fixed.csv'")


          Date  predicted_close action_label  portfolio_value
0   2022-01-03       166.433055         Hold          10000.0
1   2022-01-04       166.320852         Sell          10000.0
2   2022-01-05       166.208649         Sell          10000.0
3   2022-01-06       166.096447         Sell          10000.0
4   2022-01-07       165.984244         Sell          10000.0
5   2022-01-10       165.872041         Sell          10000.0
6   2022-01-11       165.759838         Sell          10000.0
7   2022-01-12       165.647635         Sell          10000.0
8   2022-01-13       165.535432         Sell          10000.0
9   2022-01-14       165.423229         Sell          10000.0
10  2022-01-18       165.311026         Sell          10000.0
11  2022-01-19       165.198824         Sell          10000.0
12  2022-01-20       165.086621         Sell          10000.0
13  2022-01-21       164.974418         Sell          10000.0
14  2022-01-24       164.862215         Sell          10000.0
✅ Fixed 

In [19]:
import pandas as pd

# Load SFT dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Initialize action column
df['action_label'] = 'Hold'

# Thresholds for price changes (absolute)
buy_threshold = 0.01   # $0.01 increase triggers Buy
sell_threshold = -0.01 # $0.01 decrease triggers Sell

# Assign actions based on predicted_close change
for i in range(1, len(df)):
    change = df.loc[i, 'predicted_close'] - df.loc[i-1, 'predicted_close']
    if change > buy_threshold:
        df.loc[i, 'action_label'] = 'Buy'
    elif change < sell_threshold:
        df.loc[i, 'action_label'] = 'Sell'
    else:
        df.loc[i, 'action_label'] = 'Hold'

# Simulate portfolio value
initial_cash = 10000
position = 1  # start with 1 share
cash = initial_cash - df.loc[0, 'predicted_close']  # adjust cash after buying first share
portfolio_values = []

for i in range(len(df)):
    price = df.loc[i, 'predicted_close']
    action = df.loc[i, 'action_label']

    if action == 'Buy' and cash >= price:
        position += 1
        cash -= price
    elif action == 'Sell' and position > 0:
        position -= 1
        cash += price

    portfolio_value = cash + position * price
    portfolio_values.append(portfolio_value)

df['portfolio_value'] = portfolio_values

# Display first 15 rows
print(df[['Date', 'predicted_close', 'action_label', 'portfolio_value']].head(15))

# Save to CSV
df.to_csv("aapl_rl_simulation_final.csv", index=False)
print("✅ Final RL simulation output saved to 'aapl_rl_simulation_final.csv'")


          Date  predicted_close action_label  portfolio_value
0   2022-01-03       166.433055         Hold     10000.000000
1   2022-01-04       166.320852         Sell      9999.887797
2   2022-01-05       166.208649         Sell      9999.887797
3   2022-01-06       166.096447         Sell      9999.887797
4   2022-01-07       165.984244         Sell      9999.887797
5   2022-01-10       165.872041         Sell      9999.887797
6   2022-01-11       165.759838         Sell      9999.887797
7   2022-01-12       165.647635         Sell      9999.887797
8   2022-01-13       165.535432         Sell      9999.887797
9   2022-01-14       165.423229         Sell      9999.887797
10  2022-01-18       165.311026         Sell      9999.887797
11  2022-01-19       165.198824         Sell      9999.887797
12  2022-01-20       165.086621         Sell      9999.887797
13  2022-01-21       164.974418         Sell      9999.887797
14  2022-01-24       164.862215         Sell      9999.887797
✅ Final 

In [20]:
import pandas as pd

# Load dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Initialize actions
df['action_label'] = 'Hold'

# Thresholds for price change
buy_threshold = 0.01
sell_threshold = -0.01

# Assign Buy/Sell/Hold actions
for i in range(1, len(df)):
    change = df.loc[i, 'predicted_close'] - df.loc[i-1, 'predicted_close']
    if change > buy_threshold:
        df.loc[i, 'action_label'] = 'Buy'
    elif change < sell_threshold:
        df.loc[i, 'action_label'] = 'Sell'
    else:
        df.loc[i, 'action_label'] = 'Hold'

# Simulate portfolio
initial_cash = 10000
cash = initial_cash
position = 0
portfolio_values = []

for i in range(len(df)):
    price = df.loc[i, 'predicted_close']
    action = df.loc[i, 'action_label']

    if action == 'Buy' and cash >= price:
        # Buy as many shares as possible
        shares_to_buy = int(cash // price)
        position += shares_to_buy
        cash -= shares_to_buy * price
    elif action == 'Sell' and position > 0:
        # Sell all shares
        cash += position * price
        position = 0

    portfolio_value = cash + position * price
    portfolio_values.append(portfolio_value)

df['portfolio_value'] = portfolio_values

# Show first 15 rows
print(df[['Date', 'predicted_close', 'action_label', 'portfolio_value']].head(15))

# Save to CSV
df.to_csv("aapl_rl_simulation_realistic.csv", index=False)
print("✅ Realistic RL simulation output saved to 'aapl_rl_simulation_realistic.csv'")


          Date  predicted_close action_label  portfolio_value
0   2022-01-03       166.433055         Hold          10000.0
1   2022-01-04       166.320852         Sell          10000.0
2   2022-01-05       166.208649         Sell          10000.0
3   2022-01-06       166.096447         Sell          10000.0
4   2022-01-07       165.984244         Sell          10000.0
5   2022-01-10       165.872041         Sell          10000.0
6   2022-01-11       165.759838         Sell          10000.0
7   2022-01-12       165.647635         Sell          10000.0
8   2022-01-13       165.535432         Sell          10000.0
9   2022-01-14       165.423229         Sell          10000.0
10  2022-01-18       165.311026         Sell          10000.0
11  2022-01-19       165.198824         Sell          10000.0
12  2022-01-20       165.086621         Sell          10000.0
13  2022-01-21       164.974418         Sell          10000.0
14  2022-01-24       164.862215         Sell          10000.0
✅ Realis

In [21]:
import pandas as pd

# Load dataset
df = pd.read_csv("aapl_sft_output_realistic.csv")

# Initialize actions
df['action_label'] = 'Hold'

# Portfolio simulation
initial_cash = 10000
cash = initial_cash
position = 0
portfolio_values = []

# Decide actions based on trend
for i in range(len(df)):
    trend = df.loc[i, 'trend']
    price = df.loc[i, 'predicted_close']

    if trend == 'Bullish' and cash >= price:
        # Buy as many shares as possible
        shares_to_buy = int(cash // price)
        position += shares_to_buy
        cash -= shares_to_buy * price
        df.loc[i, 'action_label'] = 'Buy'
    elif trend == 'Bearish' and position > 0:
        # Sell all shares
        cash += position * price
        position = 0
        df.loc[i, 'action_label'] = 'Sell'
    else:
        df.loc[i, 'action_label'] = 'Hold'

    # Portfolio value = cash + value of shares
    portfolio_values.append(cash + position * price)

df['portfolio_value'] = portfolio_values

# Show first 15 rows
print(df[['Date', 'predicted_close', 'trend', 'action_label', 'portfolio_value']].head(15))

# Save to CSV
df.to_csv("aapl_rl_simulation_trend_based.csv", index=False)
print("✅ Trend-based RL simulation output saved to 'aapl_rl_simulation_trend_based.csv'")


          Date  predicted_close    trend action_label  portfolio_value
0   2022-01-03       166.433055  Bearish         Hold     10000.000000
1   2022-01-04       166.320852  Bearish         Hold     10000.000000
2   2022-01-05       166.208649  Bearish         Hold     10000.000000
3   2022-01-06       166.096447  Bearish         Hold     10000.000000
4   2022-01-07       165.984244  Bearish         Hold     10000.000000
5   2022-01-10       165.872041  Neutral         Hold     10000.000000
6   2022-01-11       165.759838  Bearish         Hold     10000.000000
7   2022-01-12       165.647635  Bearish         Hold     10000.000000
8   2022-01-13       165.535432  Bearish         Hold     10000.000000
9   2022-01-14       165.423229  Bearish         Hold     10000.000000
10  2022-01-18       165.311026  Bearish         Hold     10000.000000
11  2022-01-19       165.198824  Bearish         Hold     10000.000000
12  2022-01-20       165.086621  Bullish          Buy     10000.000000
13  20

**DeepSeek CODE**